# Optimizing the CNN with TensorBoard

In [27]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
# Loading TensorBoard
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [29]:
from tensorboard.plugins.hparams import api as hp

In [30]:
import random
random.seed(123)

In [31]:
# import window_generator as WindowGenerator
from window_generator import WindowGenerator

# Configuring hyperparameters

In [32]:
# Optimizers (SGD, Adam, Adamax, RMSprop)
optimizers = [
    'adam', 
    'sgd', 
    'adamax', 
    'rmsprop'
]


# Loss functions (MeanSquaredError, Huber, MeanSquaredLogarithmicError)
loss_functions = [
    'huber', 
    'MeanSquaredError', 
    'MeanSquaredLogarithmicError'
]

In [33]:
# Hyperparameters
CONV_WIDTH = hp.HParam('CONV_WIDTH', hp.Discrete([3, 5]))
CONV_ACTIVACTION = hp.HParam('CONV_ACTIVACTION', hp.Discrete(['relu', 'linear']))
CONV_FILTERS = hp.HParam('CONV_FILTERS', hp.Discrete([128, 256]))
WINDOW_SIZE = hp.HParam('WINDOW_SIZE', hp.Discrete([12, 24, 48]))

OPTIMIZER = hp.HParam('Optimizer', hp.Discrete(optimizers))
LOSS_FUNCTION = hp.HParam('Loss_function', hp.Discrete(loss_functions))


# Metrics
METRIC_RMSE = 'RootMeanSquaredError'


with tf.summary.create_file_writer('logs/CNN').as_default():
    hp.hparams_config(
    # Parameters
    hparams = [
        CONV_WIDTH,
        CONV_ACTIVACTION, 
        CONV_FILTERS,
        WINDOW_SIZE,
        OPTIMIZER,
        LOSS_FUNCTION,  
    ],
    # Metrics
    metrics = [
        hp.Metric(
            METRIC_RMSE, display_name='RMSE'
        ),
    ]
    )

# Loading the dataset

In [7]:
df = pd.read_csv('extracted_data/csv/hour_merged.csv',
                 parse_dates={'data': ['date', 'time']},
                 infer_datetime_format=True,
                 dayfirst=False,
                 index_col='data'
                )


df.head()

,price,demand,emissions
data,,,
2017-01-01 00:00:00,58.82,24682,14467.2840
2017-01-01 01:00:00,58.23,24046,14173.8295
2017-01-01 02:00:00,51.95,22665,13198.0690
2017-01-01 03:00:00,47.27,21200,12510.7595
2017-01-01 04:00:00,45.49,20056,12203.7315


In [9]:
# Select data from 2021
df = df[int((len(df)*0.8)):]
df.describe()

,price,demand,emissions
count,8760.000000,8760.000000,8760.000000
mean,149.843062,27748.348744,9344.160081
std,105.417526,4247.456217,3377.646274
min,0.010000,17443.000000,3890.149500
25%,61.360000,24236.500000,6815.296250
50%,121.995000,27959.500000,8704.811500
75%,240.502500,30857.250000,11209.781125
max,536.810000,41552.000000,24229.262000


# Differencing

In [10]:
df.head()

,price,demand,emissions
data,,,
2021-01-01 00:00:00,50.87,25134,6356.6480
2021-01-01 01:00:00,48.19,24179,6269.1480
2021-01-01 02:00:00,44.68,22472,5702.4820
2021-01-01 03:00:00,38.50,20319,5284.9275
2021-01-01 04:00:00,36.80,19040,5218.2340


In [11]:
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.Series(diff)

In [12]:
diff_price = difference(df['price'])
diff_emissions = difference(df['emissions'])
diff_demand = difference(df['demand'])

In [13]:
s1 = pd.Series([0.0])
diff_price = s1.append(diff_price, ignore_index=True)
diff_emissions = s1.append(diff_emissions, ignore_index=True)
diff_demand = s1.append(diff_demand, ignore_index=True)

In [14]:
df['price'] = diff_price.values
df['demand'] = diff_demand.values
df['emissions'] = diff_emissions.values

In [15]:
df.head()

,price,demand,emissions
data,,,
2021-01-01 00:00:00,0.00,0.0,0.0000
2021-01-01 01:00:00,-2.68,-955.0,-87.5000
2021-01-01 02:00:00,-3.51,-1707.0,-566.6660
2021-01-01 03:00:00,-6.18,-2153.0,-417.5545
2021-01-01 04:00:00,-1.70,-1279.0,-66.6935


# Split into training and test

In [16]:
# 80% for training and 20% for test
n = len(df)
train_df = df[0:int(n*0.8)]
test_df = df[int(n*0.8):]

# Normalization/Standarization

In [17]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [18]:
train_df.head()

,price,demand,emissions
data,,,
2021-01-01 00:00:00,-0.001166,0.000057,-0.000507
2021-01-01 01:00:00,-0.121941,-0.737243,-0.163255
2021-01-01 02:00:00,-0.159345,-1.317819,-1.054494
2021-01-01 03:00:00,-0.279669,-1.662149,-0.777150
2021-01-01 04:00:00,-0.077777,-0.987385,-0.124555


# CNN

In [19]:
MAX_EPOCHS = 20
patience = 4
num_features=df.shape[1]
OUT_STEPS = 24

In [20]:
multi_val_performance = {}
multi_performance = {}

In [34]:
def train_test_model(hparams):
    
    multi_conv_model = tf.keras.Sequential([
        # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
        tf.keras.layers.Lambda(lambda x: x[:, -hparams[CONV_WIDTH]:, :]),
        # Shape => [batch, 1, conv_units]
        tf.keras.layers.Conv1D(hparams[CONV_FILTERS], activation=hparams[CONV_ACTIVACTION], kernel_size=(hparams[CONV_WIDTH])),
        # Shape => [batch, 1,  out_steps*features]
        tf.keras.layers.Dense(OUT_STEPS*num_features,
                              kernel_initializer=tf.initializers.zeros()),
        # Shape => [batch, out_steps, features]
        tf.keras.layers.Reshape([OUT_STEPS, num_features])
    ])
    
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    
    model.compile(loss=hparams[LOSS_FUNCTION],
                 optimizer=hparams[OPTIMIZER],
                 metrics=[tf.metrics.RootMeanSquaredError()])
    
    # Creating a Window
    window = WindowGenerator(input_width=hparams[WINDOW_SIZE], label_width=OUT_STEPS, shift=OUT_STEPS, train_df=train_df, test_df=test_df)

    history = model.fit(window.train, epochs=MAX_EPOCHS, validation_data = window.test, callbacks=[early_stopping])
    
    print(multi_conv_model.evaluate(window.test, verbose=0))
    loss, rmse = multi_conv_model.evaluate(window.test, verbose=0)
    
    return rmse

In [35]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        rmse = train_test_model(hparams)
        
        tf.summary.scalar(METRIC_RMSE, rmse, step=1)

In [ ]:
session_num = 0
for conv_width in CONV_WIDTH.domain.values:
    for conv_activation in CONV_ACTIVACTION.domain.values:
        for conv_filters in CONV_FILTERS.domain.values:
            for window_size in WINDOW_SIZE.domain.values:
                for optimizer in OPTIMIZER.domain.values:
                    for loss_function in LOSS_FUNCTION.domain.values:
                        hparams = {
                            CONV_WIDTH: conv_width,
                            CONV_ACTIVACTION: conv_activation, 
                            CONV_FILTERS: conv_filters,
                            WINDOW_SIZE: window_size,
                            OPTIMIZER: optimizer,
                            LOSS_FUNCTION: loss_function
                        }
                        
                        run_name = "run-%d" % session_num
                        print('--- Starting trial: %s' % run_name)
                        print({h.name: hparams[h] for h in hparams})
                        run('logs/CNN/' + run_name, hparams)
                        session_num += 1